In [1]:
import pandas as pd
import os
import numpy as np
from pathlib import Path
import math
import torch
from copy import deepcopy
from itertools import chain 
from torch.utils.data import Dataset
from torchtext.vocab import vocab as Vocab

import warnings
warnings.filterwarnings("ignore")


Pathing


In [2]:
#Lokalt
data_dir = 'c:\\Users\\erika\\Desktop\\Exjobb\\data'

In [2]:
#saga
data_dir = "/home/aeerik/data/raw/"


Data import

In [5]:
from data_preprocessing import data_loader
include_pheno = False
threshold_year = 1970
path = data_dir

NCBI = data_loader(include_pheno,threshold_year,path)
NCBI.head()

,year,location,genes
1,[PAD],USA,"[sul1, tet(A), aadA1]"
4,[PAD],USA,"[sul2, aph(3'')-Ib, tet(A), aph(6)-Id]"
5,[PAD],USA,"[glpT_E448K=POINT, pmrB_Y358N=POINT]"
6,[PAD],Sweden,"[uhpT_E350Q=POINT, cyaA_S352T=POINT, glpT_E448..."
7,1979,USA,[glpT_E448K=POINT]


Vocabulary

In [6]:
from build_vocabulary import make_vocabulary
vocabulary = make_vocabulary(NCBI)
print(len(vocabulary))

1208


In [3]:
from create_dataset import NCBIDataset
from build_vocabulary import make_vocabulary
from data_preprocessing import data_loader

include_pheno = False
threshold_year = 1970
path = data_dir

NCBI = data_loader(include_pheno,threshold_year,path)

max_length = 20
mask_prob = 0.30
vocabulary = make_vocabulary(NCBI)

test_set = NCBIDataset(NCBI, vocabulary, max_length, mask_prob)
test_set.prepare_dataset()

In [36]:
test_set[1][2]

tensor([[False,  True, False, False, False, False, False,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True]])

Embedding 

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class JointEmbedding(nn.Module):

    def __init__(self, embedding_dim, vocab_size, max_length, drop_prob):
        super(JointEmbedding, self).__init__()

        self.max_length = max_length
        self.drop_prob = drop_prob
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim

        self.token_emb = nn.Embedding(self.vocab_size, self.embedding_dim)

        self.dropout = nn.Dropout(drop_prob)	
        self.norm = nn.LayerNorm(self.embedding_dim)

    def forward(self, input_tensor):
        token_embedding = self.token_emb(input_tensor)
        token_embedding = self.norm(token_embedding)
        token_embedding = self.dropout(token_embedding)

        return token_embedding


In [5]:
from embedding import JointEmbedding
embedding_dim = 32
voca_size = len(vocabulary)
max_length = 20
drop_prob = 0.2

emb_test = JointEmbedding(embedding_dim, voca_size, max_length, drop_prob)
emb_test.forward(test_set[1][0])

tensor([[ 1.5714e+00,  1.9799e+00,  3.0669e-01, -1.3956e+00, -2.1429e+00,
          3.9344e-01,  4.3379e-02,  0.0000e+00, -0.0000e+00,  1.7810e+00,
         -3.9750e-02, -2.5747e-01,  0.0000e+00, -0.0000e+00,  0.0000e+00,
          6.0221e-01,  0.0000e+00, -7.9298e-01,  3.4760e-01, -1.2773e+00,
          1.9465e-01,  1.7789e-01,  1.1496e+00, -1.7718e+00,  0.0000e+00,
         -1.5539e+00, -6.7535e-01, -0.0000e+00, -3.0521e-01, -2.8486e-01,
          1.0055e+00, -0.0000e+00],
        [ 1.9374e+00,  1.8804e-01,  0.0000e+00, -1.0741e+00, -3.6639e+00,
         -8.4977e-01, -5.3423e-01,  1.2774e+00, -5.8289e-01,  0.0000e+00,
          2.4121e-01,  6.2379e-01,  1.9994e-01, -2.9658e-01, -0.0000e+00,
          9.3435e-01,  9.9618e-02, -9.6678e-01,  1.0044e+00, -1.7053e+00,
          0.0000e+00,  2.5176e-01,  2.9266e-03,  2.3632e+00, -2.0279e+00,
         -1.5983e+00, -4.8785e-01,  2.0786e-01,  2.5062e-01,  1.5050e+00,
         -1.2354e+00,  3.6361e-01],
        [-1.3215e+00,  1.6768e+00, -2.57

In [16]:
test_set[0][0]

tensor([  0,   1,  59, 213, 214, 215,   1,   1,   1,   1,   1,   1,   1,   1,
          1,   1,   1,   1,   1,   1])

In [6]:
import torch

from torch import nn
import torch.nn.functional as f

class AttentionHead(nn.Module):

    def __init__(self, dim_inp, dim_out, drop_prob):
        super(AttentionHead, self).__init__()

        self.dim_inp = dim_inp
        self.drop_prob = drop_prob
        self.dropout = nn.Dropout(drop_prob)
        self.q = nn.Linear(dim_inp, dim_out)
        self.k = nn.Linear(dim_inp, dim_out)
        self.v = nn.Linear(dim_inp, dim_out)

    def forward(self, input_tensor: torch.Tensor, attention_mask: torch.Tensor = None):
        query, key, value = self.q(input_tensor), self.k(input_tensor), self.v(input_tensor)

        scale = query.size(1) ** 0.5
        scores = torch.matmul(query, key.transpose(-1, -2)) / scale

        scores = scores.masked_fill_(attention_mask, -1e9)
        attn = f.softmax(scores, dim=-1)
        attn = self.dropout(attn)
        context = torch.matmul(attn, value)

        return context


class MultiHeadAttention(nn.Module):

    def __init__(self, num_heads, dim_inp, dim_out,drop_prob):
        super(MultiHeadAttention, self).__init__()

        self.heads = nn.ModuleList([
            AttentionHead(dim_inp, dim_out,drop_prob) for _ in range(num_heads)
        ])
        self.linear = nn.Linear(dim_out * num_heads, dim_inp)
        self.norm = nn.LayerNorm(dim_inp)

    def forward(self, input_tensor: torch.Tensor, attention_mask: torch.Tensor):
        s = [head(input_tensor, attention_mask) for head in self.heads]
        scores = torch.cat(s, dim=-1)
        scores = self.linear(scores)
        return self.norm(scores)
    
attention_test = AttentionHead(32, 32, 0.2)
attention_test.forward(emb_test.forward(test_set[1][0]), test_set[1][2])

multihead_test = MultiHeadAttention(8, 32, 32,0.2)
multihead_test.forward(emb_test.forward(test_set[1][0]), test_set[1][2])

tensor([[-6.6909e-01, -1.8697e-01,  9.0842e-01,  5.8276e-01, -6.8046e-01,
          3.7259e-01,  2.6446e+00,  1.1426e-01, -1.5363e+00, -2.1553e-03,
          9.0254e-01, -1.2747e+00,  1.1188e+00,  8.3679e-01, -4.2130e-01,
         -7.0474e-01, -2.9925e-01,  9.5750e-01, -6.4519e-01, -1.3527e+00,
          6.9914e-01, -1.6981e+00,  1.1297e+00, -1.0368e+00, -2.3018e-01,
         -8.1127e-01, -8.9514e-01,  9.6993e-01, -8.2642e-01,  4.0037e-01,
         -1.0679e-01,  1.7401e+00],
        [-7.2743e-01,  1.0375e+00, -6.0530e-02,  1.0154e+00, -1.3250e-01,
          8.7402e-01,  1.7653e+00, -9.7701e-01, -9.2848e-01,  2.8255e-01,
          1.3136e+00, -8.4794e-01, -2.6193e-01,  2.3415e-01, -6.7953e-01,
          8.4463e-02, -8.7608e-02,  7.2859e-01, -8.1078e-01, -1.4140e+00,
          9.1027e-01, -2.2795e+00,  6.1702e-01, -8.4930e-01, -9.8813e-01,
         -1.7791e+00, -4.7617e-01,  1.6523e+00,  4.9278e-01,  1.0183e+00,
         -7.3926e-02,  1.3477e+00],
        [-6.8208e-01,  8.3759e-02,  7.11

In [7]:
import torch

from torch import nn
import torch.nn.functional as f

#dropout 
#num heads
#dim in, dim out

class Encoder(nn.Module):

    def __init__(self, dim_inp, dim_out, attention_heads, dropout_prob):
        super(Encoder, self).__init__()
        self.dropout_prob = dropout_prob
        self.attention_heads = attention_heads
        self.dim_inp = dim_inp
        self.dim_out = dim_out
        
        
        self.attention = MultiHeadAttention(self.attention_heads, self.dim_inp, self.dim_out, self.dropout_prob)  # batch_size x sentence size x dim_inp
        self.feed_forward = nn.Sequential(
            nn.Linear(self.dim_inp, self.dim_out),
            nn.Dropout(self.dropout_prob),
            nn.GELU(),
            nn.Linear(self.dim_out, self.dim_inp),
            nn.Dropout(self.dropout_prob)
        )
        self.norm = nn.LayerNorm(self.dim_inp)

    def forward(self, input_tensor: torch.Tensor, attention_mask: torch.Tensor):
        context = self.attention(input_tensor, attention_mask)
        res = self.feed_forward(context)
        return self.norm(res)

encoder_test = Encoder(32, 32, 8, 0.2)
encoder_test.forward(emb_test.forward(test_set[1][0]), test_set[1][2])

tensor([[ 6.3525e-01, -8.8667e-02,  6.0907e-01,  6.0442e-01, -4.5398e-01,
         -5.8728e-01, -1.8084e+00,  5.8872e-01,  3.3007e-01, -7.2990e-01,
         -3.1380e+00,  2.5161e-01, -1.0176e-01, -5.0544e-01,  4.3036e-01,
         -8.8667e-02,  6.5004e-01, -1.9873e+00, -2.7945e-01,  5.3622e-01,
         -8.8667e-02, -8.8667e-02, -1.5639e-01, -2.6258e-01,  1.6255e+00,
          7.3943e-01, -5.2683e-01,  9.9894e-01, -8.8667e-02,  2.4373e+00,
         -8.8667e-02,  6.3246e-01],
        [-4.0210e-01, -1.6813e+00, -2.0633e-03, -4.5533e-01, -2.0633e-03,
          1.4712e+00, -1.1608e+00, -6.8783e-01,  1.6201e+00, -3.3210e-01,
         -1.0330e+00,  1.4123e+00,  1.0335e+00, -1.4214e+00,  1.6370e+00,
          9.3001e-01,  6.5768e-01, -1.4558e+00, -7.7938e-02,  1.3517e+00,
         -8.7531e-01,  2.4981e-01, -5.9505e-01, -5.0368e-01, -2.0633e-03,
         -4.0506e-01,  2.3409e-01,  6.6320e-01, -8.1837e-01,  2.0545e+00,
         -5.7962e-01, -8.2420e-01],
        [ 1.0674e+00, -9.9510e-01, -5.97

In [8]:
from bert_builder import AttentionHead

attention_test = AttentionHead(32, 32, 0.2)
attention_test.forward(emb_test.forward(test_set[1][0]), test_set[1][2])

tensor([[-1.7382e-01,  5.5883e-01,  1.5567e-01,  3.1990e-01,  5.7133e-01,
          4.0981e-01,  3.2692e-01, -9.8584e-02,  1.1091e-01,  6.2017e-01,
         -6.4680e-01,  7.4113e-01, -3.8412e-01, -1.3157e-01,  7.6336e-01,
          6.5113e-01,  6.5063e-01, -6.0685e-01,  4.5817e-01, -5.4367e-01,
         -2.6976e-01,  4.6390e-02,  3.5174e-01,  1.6366e-01, -7.5593e-01,
         -2.1101e-01, -1.0115e-01, -2.4117e-01, -2.1898e-01, -1.0887e-01,
         -2.7023e-01, -1.6053e-01],
        [-2.0733e-01,  6.3985e-01,  1.6843e-01,  4.9425e-01,  4.0661e-01,
          5.1661e-01,  3.1085e-01, -1.8387e-01,  3.1839e-01,  8.7926e-01,
         -6.4980e-01,  9.6707e-01, -4.4520e-01,  1.7773e-02,  1.0787e+00,
          9.5964e-01,  6.6320e-01, -6.7362e-01,  4.6321e-01, -5.9017e-01,
         -3.4319e-01, -3.6880e-02,  3.2240e-01, -2.8682e-02, -6.8955e-01,
          5.2774e-02,  5.1552e-02, -2.5093e-01, -4.4234e-01, -2.8981e-01,
         -2.5356e-01, -2.8167e-01],
        [-1.2233e-01,  5.9438e-01,  8.99

In [9]:
from bert_builder import MultiHeadAttention
multihead_test = MultiHeadAttention(num_heads=8, dim_inp=32, dim_out=32, drop_prob=0.2)
multihead_test.forward(emb_test.forward(test_set[1][0]), test_set[1][2])

tensor([[ 6.1750e-03, -1.7855e+00, -9.7728e-02, -1.2552e+00,  8.9630e-01,
         -3.7337e-01,  3.6670e-01, -8.5945e-01,  5.0441e-01, -1.7264e+00,
          8.3572e-01, -8.9990e-02,  7.8844e-01, -7.8462e-01,  7.9141e-02,
          3.5409e-01, -6.6142e-01,  1.3595e+00,  6.8191e-01,  1.0993e+00,
         -5.4793e-01,  2.4829e+00, -3.3265e-01,  1.9503e+00,  7.4729e-01,
         -1.8288e-01, -1.3283e+00, -6.0981e-01,  3.4441e-01,  3.5611e-01,
         -7.0608e-01, -1.5113e+00],
        [-4.5841e-01, -1.2943e+00,  4.1449e-01,  2.1297e-01, -5.5660e-01,
         -3.4216e-01, -2.8374e-01, -1.1869e+00,  9.0785e-01, -1.5825e+00,
          1.2108e+00, -3.6231e-01,  6.9447e-01, -7.1402e-01, -4.3356e-01,
          1.2448e+00, -7.1112e-01,  1.7440e+00,  2.9015e-01,  9.3025e-02,
         -6.7412e-01,  1.9321e+00, -6.1007e-01,  2.5739e+00,  3.1022e-01,
         -3.3573e-01, -1.6846e-02,  9.2585e-02,  4.0845e-01,  2.4300e-01,
         -6.6363e-01, -2.1468e+00],
        [ 7.2884e-01, -6.6557e-01, -1.81

In [10]:
from bert_builder import Encoder

encoder_test = Encoder(32, 32, 8, 0.2)
encoder_test.forward(emb_test.forward(test_set[1][0]), test_set[1][2])

tensor([[-1.9672e-01, -1.7521e-02, -1.2679e+00, -9.8153e-01, -1.7521e-02,
          2.1980e+00,  1.4354e+00,  3.0122e-01,  6.0957e-01,  7.9411e-01,
         -1.4883e+00,  9.8780e-01, -4.4560e-01,  9.3972e-01, -1.6206e+00,
          3.0249e-01,  1.5242e+00, -1.7521e-02, -2.1519e-02, -1.9392e+00,
          1.0905e+00, -1.0577e+00, -1.4118e+00, -8.4594e-01, -1.7521e-02,
          4.0818e-01,  6.1006e-01,  7.6525e-01, -1.2958e+00, -1.7521e-02,
         -1.7521e-02,  7.1130e-01],
        [-1.4011e+00,  3.8568e-02, -6.7254e-01, -6.8758e-01,  3.8568e-02,
          3.8568e-02,  1.7268e+00,  3.8568e-02,  3.8568e-02,  7.0799e-01,
         -2.2126e+00,  1.7930e+00,  3.8568e-02,  6.0438e-01, -2.0003e+00,
          1.0517e+00,  7.9339e-01,  1.6358e+00, -4.0570e-01,  3.8568e-02,
          1.9390e-01, -1.1961e+00, -1.1144e+00, -8.5154e-01,  3.8568e-02,
          1.0344e+00,  1.0361e+00,  3.8568e-02, -1.2849e+00, -2.5871e-01,
          3.8568e-02,  1.1224e+00],
        [-1.2142e+00,  6.1444e-01, -4.54

---------------------------

dataset
- behöver vocabulary
- saknar: getitem och prepare dataset som kallar på construct_masking

In [1]:
from torchtext.vocab import vocab 
from copy import deepcopy
from collections import Counter
import torch
from torch.utils.data import Dataset
import pandas as pd
from itertools import chain
import numpy as np

from data_preprocessing import data_loader

include_pheno = True
threshold_year = 1970

NCBI = data_loader(include_pheno,threshold_year)
NCBI.fillna('[PAD]', inplace=True)
print('data importerad')


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MASK_PERCENTAGE = 0.15


# data, vocabulary, max sequence length, mask probability, include sequences, some random state
class NCBIDataset(Dataset):

    MASKED_INDICES_COLUMN = 'masked_indices'
    TARGET_COLUMN = 'indices'
    NSP_TARGET_COLUMN = 'is_next'
    TOKEN_MASK_COLUMN = 'token_mask'

    def __init__(self,
                 data: pd.DataFrame,
                 vocab: vocab,
                 max_seq_len: int,
                 mask_prob: float,
                 random_state: int = 23,
                 ):
        
        self.random_state = random_state
        np.random.seed(self.random_state)

        CLS = '[CLS]'
        PAD = '[PAD]'
        MASK = '[MASK]'
        UNK = '[UNK]'

        self.data = data.reset_index(drop=True) 
        self.num_samples = self.data.shape[0]
        self.vocab = vocab
        self.vocab_size = len(self.vocab)
        self.CLS = CLS 
        self.PAD = PAD
        self.MASK = MASK
        self.UNK = UNK
        self.max_seq_len = max_seq_len
        self.mask_prob = mask_prob
        self.columns = [self.MASKED_INDICES_COLUMN, self.TARGET_COLUMN]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data.iloc[idx]
        input = torch.Tensor(item[self.MASKED_INDICES_COLUMN],device=device).long()
        token_mask  = torch.tensor(item[self.TARGET_COLUMN], device=device).long()
        attention_mask = (input == self.vocab[self.PAD]).unsqueeze(0)

        return input, token_mask , attention_mask

    def _construct_masking(self):
        sequences = deepcopy(self.data['genes'].tolist())
        masked_sequences = list()
        target_indices_list = list()
        seq_starts = [[self.CLS, self.data['year'].iloc[i], self.data['location'].iloc[i]] for i in range(self.data.shape[0])]

        for i, geno_seq in enumerate(sequences):
            seq_len = len(geno_seq)
            masking_index = np.random.rand(seq_len) < self.mask_prob   
            target_indices = np.array([-1]*seq_len)
            indices = masking_index.nonzero()[0]
            target_indices[indices] = self.vocab.lookup_indices([geno_seq[i] for i in indices])
            for i in indices:
                r = np.random.rand()
                if r < 0.8:
                    geno_seq[i] = self.MASK
                elif r > 0.9:
                    geno_seq[i] = self.vocab.lookup_token(np.random.randint(self.vocab_size))
            geno_seq = seq_starts[i] + geno_seq
            target_indices = [-1]*3 + target_indices.tolist() 
            masked_sequences.append(geno_seq)
            target_indices_list.append(target_indices)
        print('här är lugnt')
        print(masked_sequences[:20])
        print(target_indices_list[:20])
        masked_sequences = [seq + [self.PAD]*(self.max_seq_len - len(seq)) for seq in masked_sequences]
        print(masked_sequences[:20])
        print(range(len(target_indices_list)))
        for i in range(len(target_indices_list)):
            indices = target_indices_list[i]
            padding = [-1] * (self.max_seq_len - len(indices))
            target_indices_list[i] = indices + padding
        print('hit men inte längre ')
        return masked_sequences, target_indices_list 
        
    def prepare_dataset(self):
        masked_sequences, target_indices = self._construct_masking()
        indices_masked = [self.vocab.lookup_indices(masked_seq) for masked_seq in masked_sequences]

        rows = zip(indices_masked, target_indices)
        self.data = pd.DataFrame(rows, columns=self.columns)
        print(self.data.head())

def make_vocabulary(dataset: pd.DataFrame):
    CLS = '[CLS]'
    PAD = '[PAD]'
    MASK = '[MASK]'
    UNK = '[UNK]'

    token_list = Counter()
    data = dataset.copy()

    location_tokens = list(dict.fromkeys(list(chain(list(data['location'])))))
    year_tokens = list(dict.fromkeys(list(chain(list(data['year'])))))
    genes_tokens = list(dict.fromkeys(list(chain(*data['genes']))))
   
    token_list.update(map(str, year_tokens))
    token_list.update(map(str, location_tokens))
    token_list.update(map(str, genes_tokens))
    vocabulary = vocab(token_list,specials = [CLS, PAD, MASK, UNK])
    return vocabulary


max_length = 10
mask_prob = 0.30
vocabulary = make_vocabulary(NCBI)

test_set = NCBIDataset(NCBI, vocabulary, max_length, mask_prob)
test_set.prepare_dataset()

data importerad
här är lugnt
[['[CLS]', '[PAD]', 'USA', 'aadA1', 'sul1', 'tet(A)'], ['[CLS]', '[PAD]', 'Sweden', '[MASK]', '[MASK]', 'tet(A)', '[MASK]'], ['[CLS]', '[PAD]', 'USA', 'glpT_E448K=POINT', 'pmrB_Y358N=POINT'], ['[CLS]', '[PAD]', 'Sweden', 'glpT_E448K=POINT', 'cyaA_S352T=POINT', 'uhpT_E350Q=POINT'], ['[CLS]', '1979', 'USA', 'glpT_E448K=POINT'], ['[CLS]', '[PAD]', 'USA', 'glpT_E448K=POINT', '[MASK]', '[MASK]'], ['[CLS]', '[PAD]', 'USA', '[MASK]', 'pmrB_E123D=POINT', '[MASK]'], ['[CLS]', '[PAD]', 'USA', 'glpT_E448K=POINT', 'pmrB_Y358N=POINT'], ['[CLS]', '[PAD]', 'USA', 'glpT_E448K=POINT', 'pmrB_Y358N=POINT'], ['[CLS]', '[PAD]', 'Sweden', 'parC_A56T=POINT', 'glpT_E448K=POINT', '[MASK]', '[MASK]'], ['[CLS]', '[PAD]', 'Sweden', 'parE_I355T=POINT'], ['[CLS]', '[PAD]', 'Sweden', 'glpT_E448K=POINT'], ['[CLS]', '[PAD]', 'Sweden', "aph(3'')-Ib", 'tet(B)', 'sul2', '[MASK]'], ['[CLS]', '[PAD]', 'Indonesia', 'glpT_E448K=POINT'], ['[CLS]', '[PAD]', 'USA', 'glpT_E448K=POINT', 'pmrB_E123D=PO

In [2]:
from torchtext.vocab import vocab 
from copy import deepcopy
from collections import Counter
import torch
from torch.utils.data import Dataset
import pandas as pd
from itertools import chain
import numpy as np

from data_preprocessing import data_loader
from create_dataset import NCBIDataset

include_pheno = True
threshold_year = 1970

NCBI = data_loader(include_pheno,threshold_year)
print('data importerad')

def make_vocabulary(dataset: pd.DataFrame):
    CLS = '[CLS]'
    PAD = '[PAD]'
    MASK = '[MASK]'
    UNK = '[UNK]'

    token_list = Counter()
    data = dataset.copy()

    location_tokens = list(dict.fromkeys(list(chain(list(data['location'])))))
    year_tokens = list(dict.fromkeys(list(chain(list(data['year'])))))
    genes_tokens = list(dict.fromkeys(list(chain(*data['genes']))))
   
    token_list.update(map(str, year_tokens))
    token_list.update(map(str, location_tokens))
    token_list.update(map(str, genes_tokens))
    vocabulary = vocab(token_list,specials = [CLS, PAD, MASK, UNK])
    return vocabulary


max_length = 10
mask_prob = 0.30
vocabulary = make_vocabulary(NCBI)

test_set = NCBIDataset(NCBI, vocabulary, max_length, mask_prob)
test_set.prepare_dataset()

data importerad
här är lugnt
[['[CLS]', nan, 'USA', 'sul1', 'tet(A)', 'aadA1'], ['[CLS]', nan, 'Sweden', '[MASK]', '[MASK]', 'aph(6)-Id', '[MASK]'], ['[CLS]', nan, 'USA', 'pmrB_Y358N=POINT', 'glpT_E448K=POINT'], ['[CLS]', nan, 'Sweden', 'uhpT_E350Q=POINT', 'cyaA_S352T=POINT', 'glpT_E448K=POINT'], ['[CLS]', '1979', 'USA', 'glpT_E448K=POINT'], ['[CLS]', nan, 'USA', 'pmrB_E123D=POINT', '[MASK]', '[MASK]'], ['[CLS]', nan, 'USA', '[MASK]', 'pmrB_E123D=POINT', '[MASK]'], ['[CLS]', nan, 'USA', 'pmrB_Y358N=POINT', 'glpT_E448K=POINT'], ['[CLS]', nan, 'USA', 'pmrB_Y358N=POINT', 'glpT_E448K=POINT'], ['[CLS]', nan, 'Sweden', 'pmrB_Y358N=POINT', 'tet(A)', '[MASK]', '[MASK]'], ['[CLS]', nan, 'Sweden', 'parE_I355T=POINT'], ['[CLS]', nan, 'Sweden', 'glpT_E448K=POINT'], ['[CLS]', nan, 'Sweden', 'sul2', "aph(3'')-Ib", 'aph(6)-Id', '[MASK]'], ['[CLS]', nan, 'Indonesia', 'glpT_E448K=POINT'], ['[CLS]', nan, 'USA', 'pmrB_E123D=POINT', 'glpT_E448K=POINT'], ['[CLS]', nan, 'USA', 'aph(6)-Id', 'glpT_E448K=POINT

: 

Vocabulary

In [ ]:
import random
import typing
from collections import Counter
from pathlib import Path

import numpy as np
import pandas as pd
from itertools import chain
import torch

from tqdm import tqdm
from torch.utils.data import Dataset
from torchtext.vocab import vocab
from torchtext.data.utils import get_tokenizer 

print(list(chain(*NCBI['genes']))[:20])

mylist = list(dict.fromkeys(list(chain(*NCBI['genes']))))
print(len(mylist))
print(mylist[:20])

test = list(chain(list(NCBI['year'])))
print(test[:40])
test = list(dict.fromkeys(list(chain(list(NCBI['year'])))))
print(len(test))
print(test[:40])

test = list(chain(list(NCBI['location'])))
print(test[:20])
test = list(dict.fromkeys(list(chain(list(NCBI['location'])))))
print(len(test))
print(test[:20])

In [ ]:
import random
import typing
from collections import Counter
from pathlib import Path

import numpy as np
import pandas as pd
from itertools import chain
import torch

from tqdm import tqdm
from torch.utils.data import Dataset
from torchtext.vocab import vocab
from torchtext.data.utils import get_tokenizer 

def make_vocabulary(dataset: pd.DataFrame):
    CLS = '[CLS]'
    PAD = '[PAD]'
    SEP = '[SEP]'
    MASK = '[MASK]'
    UNK = '[UNK]'

    token_list = Counter()
    data = dataset.copy()

    location_tokens = list(dict.fromkeys(list(chain(list(data['location'])))))
    year_tokens = list(dict.fromkeys(list(chain(list(data['year'])))))
    genes_tokens = list(dict.fromkeys(list(chain(*data['genes']))))
   
    token_list.update(map(str, year_tokens))
    token_list.update(map(str, location_tokens))
    token_list.update(map(str, genes_tokens))
    print(token_list)
    vocabulary = vocab(token_list,specials = [CLS, PAD, MASK, SEP, UNK])
    print(vocabulary)
    return vocabulary
vocabulary = make_vocabulary(NCBI)


Counter({'nan': 2, '1979': 1, '2013': 1, '1996': 1, '1975': 1, '2009': 1, '1974': 1, '1998': 1, '1983': 1, '2008': 1, '1995': 1, '1989': 1, '2003': 1, '1997': 1, '1994': 1, '2001': 1, '1980': 1, '1970': 1, '1971': 1, '2007': 1, '2004': 1, '2006': 1, '1982': 1, '1991': 1, '1990': 1, '2002': 1, '1988': 1, '1987': 1, '2011': 1, '1986': 1, '1985': 1, '2010': 1, '2005': 1, '2012': 1, '1984': 1, '2000': 1, '2014': 1, '2015': 1, '1999': 1, '1992': 1, '1973': 1, '1981': 1, '1993': 1, '2016': 1, '1972': 1, '1976': 1, '1977': 1, '2017': 1, '1978': 1, '2018': 1, '2019': 1, '2020': 1, '2021': 1, '2022': 1, '2023': 1, '2024': 1, 'USA': 1, 'Sweden': 1, 'Indonesia': 1, 'Canada': 1, 'Papua New Guinea': 1, 'Japan': 1, 'China': 1, 'Germany': 1, 'India': 1, 'Bangladesh': 1, 'Brazil': 1, 'UK': 1, 'Denmark': 1, 'South Korea': 1, 'France': 1, 'Norway': 1, 'Malaysia': 1, 'Slovenia': 1, 'Thailand': 1, 'Italy': 1, 'Israel': 1, 'Austria': 1, 'Belgium': 1, 'Guinea-Bissau': 1, 'Australia': 1, 'Hungary': 1, 'Leban

In [ ]:
vocabulary = make_vocabulary(NCBI)
